In [88]:
import os


In [89]:
%pwd


'D:\\ML Projects\\KidneyDiseaseDL\\KidneyDiseaseEndtoEndDL'

In [90]:
os.chdir("../")


In [91]:
%pwd


'D:\\ML Projects\\KidneyDiseaseDL'

In [92]:


import sys
import os

sys.path.append(os.path.abspath("D:/ML Projects/KidneyDiseaseDL/KidneyDiseaseEndtoEndDL/src"))



In [93]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [94]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [95]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [96]:

import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [97]:

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        model_path = Path(self.config.updated_base_model_path)
            # Fix path relative to project root if needed
        if not model_path.is_absolute():
            PROJECT_ROOT = Path("D:/ML Projects/KidneyDiseaseDL/KidneyDiseaseEndtoEndDL")
            model_path = PROJECT_ROOT / model_path
        print("📍 Loading model from:", model_path)
        print("✅ Model file exists:", model_path.exists())
        print("📂 Absolute path:", model_path.resolve())

        if not model_path.exists():
            raise FileNotFoundError(f"Model file not found at: {model_path}")
    

        self.model = tf.keras.models.load_model(str(model_path.resolve()))

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [98]:
%pip install Pillow


Note: you may need to restart the kernel to use updated packages.


In [99]:
os.getcwd()

'D:\\ML Projects\\KidneyDiseaseDL'

In [100]:
os.chdir(r"D:\ML Projects\KidneyDiseaseDL\KidneyDiseaseEndtoEndDL")

In [105]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [107]:


from tensorflow.keras.preprocessing.image import load_img
import PIL
print(PIL.__version__)
from PIL import Image

img = load_img("path_to_a_sample_image.jpg")
print(img.size)

10.4.0
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\ASUS\miniconda3\envs\kidney\lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\ASUS\AppData\Local\Temp\ipykernel_60584\2995512689.py", line 6, in <module>
    img = load_img("path_to_a_sample_image.jpg")
  File "c:\Users\ASUS\miniconda3\envs\kidney\lib\site-packages\keras\utils\image_utils.py", line 414, in load_img
    raise ImportError(
ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\ASUS\miniconda3\envs\kidney\lib\site-packages\IPython\core\interactiveshell.py", line 2105, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\ASUS\miniconda3\envs\kidney\lib\site-packages\IPython\core\ultratb.py", line 1396, in structured_traceback
    return FormattedTB.struc

In [102]:
from pathlib import Path
import PIL
print(PIL.__version__)

model_path = Path("artifacts/prepare_base_model/base_model_updated.h5")
print("Model path exists:", model_path.exists())
print("Resolved path:", model_path.resolve())
try:
    config = ConfigurationManager(
        config_filepath=Path("D:/ML Projects/KidneyDiseaseDL/KidneyDiseaseEndtoEndDL/config/config.yaml"),
        params_filepath=Path("D:/ML Projects/KidneyDiseaseDL/KidneyDiseaseEndtoEndDL/params.yaml")
)
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()

except Exception as e:
    raise e


10.4.0
Model path exists: True
Resolved path: D:\ML Projects\KidneyDiseaseDL\KidneyDiseaseEndtoEndDL\artifacts\prepare_base_model\base_model_updated.h5
[2025-07-28 11:38:42,857: INFO: common: yaml file: D:\ML Projects\KidneyDiseaseDL\KidneyDiseaseEndtoEndDL\config\config.yaml loaded successfully:]
[2025-07-28 11:38:42,859: INFO: common: yaml file: D:\ML Projects\KidneyDiseaseDL\KidneyDiseaseEndtoEndDL\params.yaml loaded successfully:]
[2025-07-28 11:38:42,861: INFO: common: created directory at: artifacts:]
[2025-07-28 11:38:42,861: INFO: common: created directory at: artifacts\training:]
📍 Loading model from: D:\ML Projects\KidneyDiseaseDL\KidneyDiseaseEndtoEndDL\artifacts\prepare_base_model\base_model_updated.h5
✅ Model file exists: True
📂 Absolute path: D:\ML Projects\KidneyDiseaseDL\KidneyDiseaseEndtoEndDL\artifacts\prepare_base_model\base_model_updated.h5
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
Unexpected exception formatting exception. Fal

Traceback (most recent call last):
  File "c:\Users\ASUS\miniconda3\envs\kidney\lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\ASUS\AppData\Local\Temp\ipykernel_60584\1174667042.py", line 20, in <module>
    raise e
  File "C:\Users\ASUS\AppData\Local\Temp\ipykernel_60584\1174667042.py", line 17, in <module>
    training.train()
  File "C:\Users\ASUS\AppData\Local\Temp\ipykernel_60584\1949950588.py", line 78, in train
    self.model.fit(
  File "c:\Users\ASUS\miniconda3\envs\kidney\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "c:\Users\ASUS\miniconda3\envs\kidney\lib\site-packages\keras\utils\image_utils.py", line 414, in load_img
    raise ImportError(
ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.

During handling of the above exception, another exception occurred:

Tr